In [1]:
import copy
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource
from pychunkedgraph.graph.utils.inspect import dfs_print_node

In [2]:
ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

# edges = np.array([[94803535283198313, 94803466563722624]],dtype=np.uint64)
# between 8
edges = np.array([[94524946524577880, 94595315268752176]],dtype=np.uint64)

print(cg.get_cross_chunk_edges_layer(edges))
print(cg.get_parents(edges[0]))
roots_old = cg.get_roots(edges[0])
print(roots_old)

[8]
[166582540562334005 166652909306511714]
[864691139327226349 864691139527061485]


In [7]:
nodes = np.array([525302675286511348, 525232306542348059])
cg._get_bounding_l2_children(nodes)

{525302675286511348: array([], dtype=uint64),
 525232306542348059: array([], dtype=uint64)}

In [8]:
nodes = np.array([166512309257110024, 166512309257110018, 166512240537633089])
nodes = np.array([166582540562334005, 166512171818156502])
acx = cg.get_atomic_cross_edges(nodes)

for k,v in acx.items():
    print(k)
    for i,j in v.items():
        print(i, len(j))
    print()

166582540562334005
2 4
8 28

166512171818156502
2 4
7 5



In [ ]:
from pychunkedgraph.graph.edits import add_edges
from pychunkedgraph.graph.utils.context_managers import TimeIt

edges = np.array(edges, dtype=np.uint64)
with TimeIt("add_edge"):
    done, roots = add_edges(cg, atomic_edges=edges)
    
print()
print(roots)

In [ ]:
# dfs_print_node(cg, roots_old[1], limit=2, sep="    ")

In [ ]:
limit = 1
leaves = []
for root in [*roots_old, roots[1]]:
    print(root)
    leaves.append([])
    children = np.array([root], dtype=np.uint64)
    mask = cg.get_chunk_layers(children) > limit
    while np.any(mask):
        children = cg.get_children(children[mask], flatten=True)
        mask = cg.get_chunk_layers(children) > limit
        leaves[-1].append(children[~mask])
    leaves[-1] = np.concatenate(leaves[-1])
    print(len(leaves[-1]))
    
print()    
print([len(x) for x in leaves])
    
old = np.unique(np.concatenate([leaves[0], leaves[1]]))
new = np.unique(leaves[2])
print(len(old), len(new))

In [ ]:
print(len(np.setdiff1d(old, new)))
diff = np.setdiff1d(old, new)
roots = cg.get_parents(diff)
print(np.unique(roots, return_counts=True))